# Crawling Website PTA Trunojoyo

Crawling (atau web crawling) adalah proses otomatis untuk mengambil informasi dari berbagai halaman web dengan cara mengakses dan mengekstrak data dari berbagai situs secara sistematis. Tujuan dari crawling adalah untuk mengumpulkan informasi yang dapat digunakan untuk berbagai keperluan, seperti analisis data, pemantauan, pengindeksan, dan lain-lain.

Crawling dilakukan oleh program komputer yang disebut "crawler" atau "spider". Crawler ini mengunjungi halaman-halaman web, mengambil konten, mengikuti tautan, dan terus bergerak di seluruh situs web untuk mengumpulkan data. Proses crawling sering digunakan dalam pengembangan mesin pencari seperti Google, analisis data web, riset pasar, dan banyak aplikasi lainnya.

## 1. Install Library BeautifulSoup

Library Beautiful Soup adalah salah satu alat yang sering digunakan untuk melakukan crawling dan ekstraksi data dari dokumen HTML dan XML. Beautiful Soup menyediakan cara yang mudah untuk menavigasi dan memanipulasi struktur dokumen HTML. Ini memungkinkan Anda melakukan pencarian elemen berdasarkan tag, atribut, dan teks yang ada di dalam elemen.

In [1]:
!pip install beautifulsoup4

## 2. Import Library

In [2]:
from bs4 import BeautifulSoup
import requests
import csv

## 3. Create Crawling Function

### 3.1. Fungsi get_data()

Fungsi ini mengambil data dari halaman tugas akhir yang diberikan URL-nya. Ia mengekstrak judul tugas akhir, nama penulis, nama dosen pembimbing, dan abstrak dari halaman tersebut.

In [3]:
def get_data(url):
  data_page = []
  response = requests.get(url)
  if response.status_code == 200:
    page = BeautifulSoup(response.content)
    container_content = page.find('li', {"data-id" : "id-1"})
    header_page = container_content.find('div', {"style" : "float:left; width:540px;"})

    #  Mengambil judul TA
    get_title = header_page.find('a', class_ = 'title').text.strip().split('\r\n')
    title = " ".join(get_title)
    data_page.append(title)

    # Mengambil Penulis dan juga Dospem
    creator = header_page.find_all('span')
    for i in creator:
      splitting_text = i.text.strip().split(':')
      if splitting_text[0].strip() == 'Penulis':
        data_page.append(splitting_text[1].strip())
      elif splitting_text[0].strip() == 'Dosen Pembimbing I':
        data_page.append(splitting_text[1].strip())
      else:
        data_page.append(splitting_text[1].strip())

    # Mengambil Abstrak
    get_abstrak = container_content.find('p', {"align" : "justify"}).text.strip().split('\r\n')
    abstrak = " ".join(get_abstrak)
    data_page.append(abstrak)

  return data_page

### 3.2. Fungsi crawling_pta()

Fungsi ini melakukan crawling atau pengambilan data dari halaman daftar tugas akhir. Ia mengambil URL halaman, mencari setiap elemen daftar tugas akhir, mengekstrak URL untuk halaman detail setiap tugas akhir, dan menggunakan fungsi `get_data()` untuk mendapatkan informasi lebih lanjut.

In [4]:
def crawling_pta(url):
  hasil_crawling = []
  response = requests.get(url)
  if response.status_code == 200:
    page = BeautifulSoup(response.content)
    container_content = page.find('ul', class_= 'items')
    list_content = container_content.find_all('li')

    for content in list_content:
      view_button = content.find('a', class_ = 'gray button').get('href')
      hasil_crawling.append(get_data(view_button))

    next_page = page.find_all('a', class_='pag_button')
    for i in next_page:
      if i.text.strip() == '>':
        return (hasil_crawling, i.get('href'))

### 3.3. Fungsi main()

Fungsi utama yang mengatur alur scraping. Ia memulai proses crawling dari URL yang diberikan sebanyak `num_page` halaman. Kemudian, ia menyimpan hasil crawling ke dalam file CSV.

In [5]:
def main(url, num_page):
  content, next_page = crawling_pta(url)

  for i in range(num_page-1):
    more_content, next_page = crawling_pta(next_page)
    content += more_content

  with open('crawling_pta.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Judul', 'Penulis', 'Dosen Pembimbing I', 'Dosen Pembimbing II', 'Abstrak'])
    writer.writerows(content)

  print(f'Data Berhasil Di Simpan Kedalam File crawling_pta.csv Dengan Jumlah : {len(content)} Data')

### 3.4. Memanggil Fungsi main()

Bagian ini menjalankan fungsi `main` dengan memberikan URL halaman pertama dan jumlah halaman yang ingin diambil.

In [6]:
main('https://pta.trunojoyo.ac.id/c_search/byprod/10', 1)

Data Berhasil Di Simpan Kedalam File crawling_pta.csv Dengan Jumlah : 500 Data


Hasil crawling (judul, penulis, dosen pembimbing, abstrak) disimpan dalam file CSV dengan nama "crawling_pta.csv". Setiap baris mewakili satu tugas akhir.